In [29]:
import uuid
from datetime import datetime

### ========== Version 2 Azure SDK
from azure.ai.ml import command, MLClient, Input, Output, dsl
from azure.ai.ml.entities import ManagedOnlineEndpoint, RecurrenceTrigger, \
    RecurrencePattern, JobSchedule
### ========== Version 1 Azure SDK
from azure.identity import InteractiveBrowserCredential
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.pipeline.core import TimeZone

subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "CV5"

config_path = "config.json"
compute_target = "adsai1"
env_name = "cv5-env-simona"
ver = 7
auth = InteractiveLoginAuthentication()
credentials = InteractiveBrowserCredential()

workspace = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name, auth=auth)
ml_client = MLClient(credential=credentials, subscription_id=subscription_id,resource_group_name=resource_group,workspace_name=workspace_name)

env = ml_client.environments.get(env_name, ver)

# Input data

In [11]:
images_all = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/main_images", display_name="train raw images")
masks_all = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/main_masks", display_name="train raw masks")
images_test = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/images", display_name="test images")
masks_test = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/sample_patches/sample_8/masks", display_name="test masks")
predict_set = Input(path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/CV5/datastores/workspaceblobstore/paths/test_set_raw_images", display_name="testset")

# Data Preprocess component

In [34]:
env = ml_client.environments.get(env_name, 6)
patchify_component = command(
    name="patchify",
    display_name="patchifying cropped data",
    description="creating patches and saving them in directories ",
    inputs=dict(
        cropped_images = Input(type="uri_folder", description='Data asset URI',mode="rw_mount"),
        type_image = "images",
        cropped_masks = Input(type="uri_folder", description='Data asset URI', mode="rw_mount"),
        type_masks = "masks"),
    outputs=dict(
        patched_images = Output(description='Data asset URI', mode="rw_mount"),
        patched_masks = Output(description='Data asset URI', mode="rw_mount")),
    code="azure/src",
    command="python cloud_preprocessing.py patchify --cropped-images-path ${{inputs.cropped_images}} --cropped-masks-path ${{inputs.cropped_masks}} --images-patch-path ${{outputs.patched_images}} --masks-patch-path ${{outputs.patched_masks}} --type-images ${{inputs.type_image}} --type-masks ${{inputs.type_masks}}",

    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1)

patchify_component = ml_client.create_or_update(patchify_component.component)
print(f"Patchify component {patchify_component.name} submitted")

Cropping component cropping submitted


# Training component

In [37]:
env = ml_client.environments.get(env_name, 6)
train_component = command(
    name="training",
    display_name="training component",
    description="train model from data assets training component",
    inputs=dict(
        epochs=1,
        train_images=Input(type="uri_folder", description='Data asset URI'),
        train_masks=Input(type="uri_folder", description='Data asset URI'),
        model_name='main_model_pipeline'),
    outputs=dict(
        model_path=Output(type="uri_folder", mode="rw_mount")),
    code="azure/src",
    command="python cloud_modelling.py train --epochs ${{inputs.epochs}} --train-images-path ${{inputs.train_images}} --train-masks-path ${{inputs.train_masks}} --model-path ${{outputs.model_path}} --model-name ${{inputs.model_name}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1)

training_component = ml_client.create_or_update(train_component.component)
print(f"Training component {training_component.name} submitted")

Patchify component datagen submitted


# Evaluating component

In [15]:
env = ml_client.environments.get(env_name, 6)
eval_component = command(
    name="eval",
    display_name="evaluating component",
    description="predicting test images from training component output",
    inputs=dict(
        test_images=Input(type="uri_folder", mode="rw_mount"),
        test_masks=Input(type="uri_folder", mode="rw_mount"),
        model_path=Input(type="uri_folder", mode="rw_mount"),
        model_name="main_model_pipeline"),
    code="azure/src",
    command="python cloud_modelling.py eval --test-images-path ${{inputs.test_images}} --test-masks-path ${{inputs.test_masks}} --model-path ${{inputs.model_path}} --model-name ${{inputs.model_name}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1,)

eval_component = ml_client.create_or_update(eval_component.component)
print(f"Evaluating component {eval_component.name} submitted")

Evaluating component eval submitted


# Predicting component

In [20]:
env = ml_client.environments.get(env_name, 6)
predict_component = command(
    name="predict",
    display_name="predicting from registered model simona",
    description="predicting test set from training component output",
    inputs=dict(
        test_images=Input(type="uri_folder", mode="rw_mount"),
        model_path=Input(type="uri_folder", mode="rw_mount"),
        model_name="main_model_pipeline"),
    outputs=dict(
        predictions=Output(type="uri_folder", mode="rw_mount")
    ),
    code="azure/src",
    command="python cloud_modelling.py predict --test-images-path ${{inputs.test_images}} --predictions ${{outputs.predictions}} --model-path ${{inputs.model_path}} --model-name ${{inputs.model_name}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1,)

predict_component = ml_client.create_or_update(predict_component.component)
print(f"Predicting component {predict_component.name} submitted")

Evaluating component predicting_simona submitted


# Return to original size

In [41]:
env = ml_client.environments.get(env_name, 6)
revert_component = command(
    name="revert",
    display_name="return to original size",
    description="return to original size",
    inputs=dict(
        predictions=Input(type="uri_folder", mode="rw_mount")),
    outputs=dict(
        output_preds=Output(type="uri_folder", mode="rw_mount")
    ),
    code="azure/src",
    command="python cloud_modelling.py revert-size --predictions ${{inputs.predictions}} --modified-preds ${{outputs.output_preds}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1,)

revert_component = ml_client.create_or_update(revert_component.component)
print(f"Reverting size component {revert_component.name} submitted")

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Reverting size component revert-env10 submitted


# Overlaying masks

In [49]:
#  modelling.overlay(args.test_images_path,args.predictions, args.modified_preds)
env = ml_client.environments.get(env_name, 11)
overlay_component = command(
    name="overlay",
    display_name="overlaying masks",
    description="overlaying predicted masks onto original test images",
    inputs=dict(
        test_images=predict_set,
        predictions=Input(type="uri_folder", mode="rw_mount")),
    outputs=dict(
        modified_preds=Output(type="uri_folder", mode="rw_mount"),
    ),
    code="azure/src",
    command="python cloud_modelling.py overlay --test-images-path ${{inputs.test_images}} --predictions ${{inputs.predictions}} --modified-preds ${{outputs.modified_preds}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1,)

overlay_component = ml_client.create_or_update(overlay_component.component)
print(f"Overlaying component {overlay_component.name} submitted")

Overlaying component overlay submitted



# Detect root tips component

In [51]:
env = ml_client.environments.get(env_name, 11)
detect_component = command(
    name="landmarks",
    display_name="landmarks detection",
    description="detecting root tips from the predicted masks",
    inputs=dict(
        predictions=Input(type="uri_folder", mode="rw_mount"),
        test_images=predict_set),
    outputs=dict(
        modified_preds=Output(type="uri_folder", mode="rw_mount"),
    ),
    code="azure/src",
    command="python cloud_modelling.py detect --test-images-path ${{inputs.test_images}} --predictions ${{inputs.predictions}} --modified-preds ${{outputs.modified_preds}}",
    environment=env,
    compute=compute_target,
    instance_type="gpu",
    instance_count=1,)

detect_component = ml_client.create_or_update(detect_component.component)
print(f"Root landmarks component {detect_component.name} submitted")

Root landmarks component landmarks submitted
